In [ ]:
from openai import AzureOpenAI
import pandas as pd
import re
import time
import csv
import json
import os
import random
import string
import os
import spacy

In [ ]:
client = AzureOpenAI(
    azure_endpoint = " ",
    api_key="<your_api_key>",
    api_version="<date>"
) 

# Read the source data

In [ ]:
#df = pd.read_csv("NumerSense.tsv", delimiter='\t', names=["Question", "Answer"])

#df = pd.read_csv("CosmosQA.csv")

#df = pd.read_json('Social_IQA/dev.jsonl', lines=True)

#df = pd.read_json('RiddleSense.jsonl', lines=True)
#df['question_stem'] = df['question'].apply(lambda x: x['stem'])

#df = pd.read_csv("ETHICS_test_hard.csv")

#df = pd.read_json('CommonsenseQA.jsonl', lines=True)
#df['question_stem'] = df['question'].apply(lambda x: x['stem'])

#df = pd.read_json('LogiQA/MRC.txt', lines=True)

#df = pd.read_json('LogiQA/NLI.txt', lines=True)

#df = pd.read_json('QASC.jsonl', lines=True)
#df['question_stem'] = df['question'].apply(lambda x: x['stem'])

#df = pd.read_json('PIQA/valid.jsonl', lines=True)

#with open('ReClor.json', 'r') as file:
 #   data = json.load(file)
#df = pd.DataFrame(data)

#df = pd.read_csv("ART.csv")
#df['combined_observations'] = df['observation_1'] + ' ' + df['observation_2']

#df = pd.read_json("GSM8K.jsonl", lines=True)

#df1 = pd.read_csv("AQuA_RAT/test.csv")
#df2 = pd.read_csv("AQuA_RAT/val.csv")
#df = pd.concat([df1, df2])

#df1 = pd.read_csv("TRAM/typical_time_mcq.csv")
#df2 = pd.read_csv("TRAM/typical_time_shots_mcq.csv")
#df = pd.concat([df1, df2])
#df = df.loc[df['Category'] == 'Commonsense']

#df = pd.read_csv("MMLU/elementary_mathematics_test.csv", header=None, names=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])

In [ ]:
'''
# LogiQA/NLI
df = df[['label', 'major_premise', 'conclusion', 'minor_premise']]
def remove_empty_list_rows(df, columns):
    for column in columns:
        df = df[df[column].apply(lambda x: len(x) != 0)]
    return df

# Columns to check for empty lists
columns_to_check = ['major_premise', 'minor_premise']

# Remove rows with empty lists in specified columns
cleaned_df = remove_empty_list_rows(df, columns_to_check)
cleaned_df['major_premise'] = cleaned_df['major_premise'].apply(lambda x: x[0] if isinstance(x, list) else x)
'''

# Red Herrings with GPT-4o

In [ ]:
# general datasets (except MMLU) for Red Herrings
system_prompt = "You are an expert in creating perturbations. For the provided text, generate Red Herrings—contextually plausible but irrelevant information—that do not alter the final answer."
generated_red_hearrings = []
for i, row in df.iterrows():
    user_prompt = f"""Given the statement: '{row['context']}', generate a single Red Herring (contextually plausible but irrelevant information) either before, after, or within the original text to challenge the LLMs while keeping the original text and final answer intact. Ensure that the Red Herring does not provide or hint at the masked information or the final answer. Output the statement with Red Herrings only."""
    #user_prompt = f"""Given the goal: '{row['goal']}' and solutions: '{row['sol1']}' and '{row['sol2']}', generate a single Red Herring (contextually plausible but irrelevant information) either before, after, or within the goal statement to challenge the LLMs while keeping the original goal and solutions intact. Output the goal with Red Herrings only."""
    #user_prompt = f"""Given the math problem: '{row['Question']}', generate a single Red Herring (contextually plausible but irrelevant information) either before, after, or within the original text. The red herring should be a detail that is not relevant to the calculations or logic required to solve the problem, but fits naturally within the context of the problem. Ensure the original text and final answer remain intact. Output the math problem with Red Herrings only."""
    try:
        response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ])
        content = response.choices[0].message.content
        time.sleep(0.1)
        if i % 50 == 0:
            print('Progress:', i, 'finished')
    except Exception as e:
        print('Error:', i, str(e)[:8])
        content = None
        
    generated_red_hearrings.append(content)

df['context_red_herrings'] = generated_red_hearrings

In [ ]:
# MMLU for Red Herrings
directory = 'MMLU'

# Function to process a single CSV file
def process_csv(file_path):
    df = pd.read_csv(file_path, header=None, names=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])
    system_prompt = "You are an expert in creating perturbations. For the provided text, generate Red Herrings—contextually plausible but irrelevant information—that do not alter the final answer."
    generated_red_herrings = []
    
    for i, row in df.iterrows():
        user_prompt = f"""Given the statement: '{row['Question']}', generate a single Red Herring (contextually plausible but irrelevant information) either before, after, or within the original text to challenge the LLMs while keeping the original text and final answer intact. Ensure that the Red Herring does not provide or hint at the masked information or the final answer. Output the statement with Red Herrings only."""
        
        try:
            response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ])
            content = response.choices[0].message.content
            time.sleep(0.1)
            if i % 50 == 0:
                print('Progress:', i, 'finished')
        except Exception as e:
            print('Error:', i, str(e)[:8])
            content = None
        
        generated_red_herrings.append(content)
    
    df['Question_red_herrings'] = generated_red_herrings
    
    # Generate the output file path
    base_name = os.path.basename(file_path)
    output_file_name = base_name.replace('_test.csv', '_RH.csv')
    output_file_path = os.path.join('MMLU_RH', output_file_name)
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    df.to_csv(output_file_path, index=False)
    print(f"Processed {file_path} and saved results to {output_file_path}")

# List all CSV files in the directory
#all_files = os.listdir(directory)
#csv_files = [file for file in all_files if file.endswith('_test.csv')]
csv_files = ['moral_scenarios_test.csv',
 'us_foreign_policy_test.csv',
 'public_relations_test.csv',
 'global_facts_test.csv',
 'electrical_engineering_test.csv',
 'astronomy_test.csv',
 'business_ethics_test.csv',
 'jurisprudence_test.csv',
 'high_school_chemistry_test.csv',
 'college_physics_test.csv',
 'professional_psychology_test.csv',
 'marketing_test.csv',
 'management_test.csv',
 'virology_test.csv',
 'international_law_test.csv',
 'high_school_macroeconomics_test.csv',
 'prehistory_test.csv',
 'abstract_algebra_test.csv',
 'high_school_physics_test.csv',
 'formal_logic_test.csv',
 'college_medicine_test.csv',
 'high_school_us_history_test.csv',
 'moral_disputes_test.csv',
 'high_school_european_history_test.csv',
 'clinical_knowledge_test.csv',
 'world_religions_test.csv',
 'high_school_microeconomics_test.csv',
 'professional_law_test.csv',
 'human_aging_test.csv',
 'medical_genetics_test.csv',
 'high_school_geography_test.csv',
 'high_school_government_and_politics_test.csv',
 'anatomy_test.csv',
 'sociology_test.csv',
 'logical_fallacies_test.csv',
 'high_school_computer_science_test.csv',
 'miscellaneous_test.csv',
 'high_school_world_history_test.csv',
 'professional_medicine_test.csv',
 'high_school_biology_test.csv',
 'high_school_statistics_test.csv',
 'college_chemistry_test.csv',
 'nutrition_test.csv',
 'econometrics_test.csv',
 'human_sexuality_test.csv',
 'security_studies_test.csv',
 'philosophy_test.csv',
 'elementary_mathematics_test.csv',
 'college_biology_test.csv',
 'college_computer_science_test.csv',
 'machine_learning_test.csv',
 'professional_accounting_test.csv',
 'college_mathematics_test.csv',
 'high_school_mathematics_test.csv',
 'high_school_psychology_test.csv',
 'conceptual_physics_test.csv',
 'computer_security_test.csv']

# Iterate over each CSV file and process it
for file_name in csv_files:
    file_path = os.path.join(directory, file_name)
    process_csv(file_path)

# Typos

In [ ]:
# general datasets (except MMLU) for typos
def swap_characters(word):
    if len(word) < 2:
        return word
    idx = random.randint(0, len(word) - 2)
    return word[:idx] + word[idx+1] + word[idx] + word[idx+2:]

def insert_random_character(word):
    idx = random.randint(0, len(word))
    random_char = random.choice(string.ascii_letters + string.digits + string.punctuation)
    return word[:idx] + random_char + word[idx:]

def delete_random_character(word):
    if len(word) < 2:
        return word
    idx = random.randint(0, len(word) - 1)
    return word[:idx] + word[idx+1:]

def replace_random_character(word):
    if len(word) < 1:
        return word
    idx = random.randint(0, len(word) - 1)
    random_char = random.choice(string.ascii_letters + string.digits + string.punctuation)
    return word[:idx] + random_char + word[idx+1:]

def apply_typo(word):
    typo_type = random.choice(['swap', 'insert', 'delete', 'replace'])
    
    if typo_type == 'swap':
        return swap_characters(word)
    elif typo_type == 'insert':
        return insert_random_character(word)
    elif typo_type == 'delete':
        return delete_random_character(word)
    elif typo_type == 'replace':
        return replace_random_character(word)

def create_typos(text, min_typos=3, max_typos=6):
    if not isinstance(text, str):
        return text  # Return the original value if it's not a string
    words = text.split()
    num_typos = random.randint(min_typos, max_typos)
    num_typos = min(num_typos, len(words))  # Ensure we don't attempt more typos than there are words
    
    typo_indices = random.sample(range(len(words)), num_typos)
    for idx in typo_indices:
        words[idx] = apply_typo(words[idx])
    
    return ' '.join(words)

# Load the CSV file
csv_file_path = 'ReClor_RH.csv'  # Replace with your actual file path
df = pd.read_csv(csv_file_path)

# Apply typos to the text column
text_column = 'context'  # Replace with the name of your text column
df['context_typos'] = df[text_column].apply(lambda x: create_typos(x, min_typos=3, max_typos=6))

df.to_csv("ReClor_RH.csv", index=False)

In [ ]:
# MMLU for typos
directory = 'MMLU'

# Function to process a single CSV file
def process_csv(file_path):
    df = pd.read_csv(file_path, header=None, names=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])
    df['Question_typos'] = df['Question'].apply(lambda x: create_typos(x, min_typos=3, max_typos=6))
    
    # Generate the output file path
    base_name = os.path.basename(file_path)
    output_file_name = base_name.replace('_test.csv', '_Typos.csv')
    output_file_path = os.path.join('MMLU_Typos', output_file_name)
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    df.to_csv(output_file_path, index=False)
    print(f"Processed {file_path} and saved results to {output_file_path}")

# List all CSV files in the directory
#all_files = os.listdir(directory)
#csv_files = [file for file in all_files if file.endswith('_test.csv')]
csv_files = ['moral_scenarios_test.csv',
 'us_foreign_policy_test.csv',
 'public_relations_test.csv',
 'global_facts_test.csv',
 'electrical_engineering_test.csv',
 'astronomy_test.csv',
 'business_ethics_test.csv',
 'jurisprudence_test.csv',
 'high_school_chemistry_test.csv',
 'college_physics_test.csv',
 'professional_psychology_test.csv',
 'marketing_test.csv',
 'management_test.csv',
 'virology_test.csv',
 'international_law_test.csv',
 'high_school_macroeconomics_test.csv',
 'prehistory_test.csv',
 'abstract_algebra_test.csv',
 'high_school_physics_test.csv',
 'formal_logic_test.csv',
 'college_medicine_test.csv',
 'high_school_us_history_test.csv',
 'moral_disputes_test.csv',
 'high_school_european_history_test.csv',
 'clinical_knowledge_test.csv',
 'world_religions_test.csv',
 'high_school_microeconomics_test.csv',
 'professional_law_test.csv',
 'human_aging_test.csv',
 'medical_genetics_test.csv',
 'high_school_geography_test.csv',
 'high_school_government_and_politics_test.csv',
 'anatomy_test.csv',
 'sociology_test.csv',
 'logical_fallacies_test.csv',
 'high_school_computer_science_test.csv',
 'miscellaneous_test.csv',
 'high_school_world_history_test.csv',
 'professional_medicine_test.csv',
 'high_school_biology_test.csv',
 'high_school_statistics_test.csv',
 'college_chemistry_test.csv',
 'nutrition_test.csv',
 'econometrics_test.csv',
 'human_sexuality_test.csv',
 'security_studies_test.csv',
 'philosophy_test.csv',
 'elementary_mathematics_test.csv',
 'college_biology_test.csv',
 'college_computer_science_test.csv',
 'machine_learning_test.csv',
 'professional_accounting_test.csv',
 'college_mathematics_test.csv',
 'high_school_mathematics_test.csv',
 'high_school_psychology_test.csv',
 'conceptual_physics_test.csv',
 'computer_security_test.csv']
# Iterate over each CSV file and process it
for file_name in csv_files:
    file_path = os.path.join(directory, file_name)
    process_csv(file_path)

# Homophones

In [ ]:
# general datasets (except MMLU) for homophones
import nltk
from nltk.corpus import cmudict

# Download the CMU Pronouncing Dictionary
nltk.download('cmudict')

# Load the CMU Pronouncing Dictionary
prondict = cmudict.dict()

def find_homophones(word):
    word = word.lower()
    homophones = []
    if word in prondict:
        word_pron = prondict[word]
        for w, pron in prondict.items():
            if w != word and pron == word_pron:
                homophones.append(w)
    return homophones

def replace_with_homophones(sentence):
    if not isinstance(sentence, str):
        return sentence 
    words = sentence.split()
    perturbed_sentence = []

    for word in words:
        homophones = find_homophones(word)
        if homophones:
            # Replace the word with a randomly chosen homophone
            new_word = random.choice(homophones)
            perturbed_sentence.append(new_word)
        else:
            perturbed_sentence.append(word)

    return ' '.join(perturbed_sentence)

# Load the CSV file
csv_file_path = 'ReClor_RH.csv'  # Replace with your actual file path
df = pd.read_csv(csv_file_path)

# Apply typos to the text column
text_column = 'context'  # Replace with the name of your text column
df['context_HP'] = df[text_column].apply(lambda x: replace_with_homophones(x))
df.to_csv("ReClor_RH.csv", index=False)

In [ ]:
# MMLU for homophones
directory = 'MMLU'

# Function to process a single CSV file
def process_csv(file_path):
    df = pd.read_csv(file_path, header=None, names=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])
    df['Question_HP'] = df['Question'].apply(lambda x: replace_with_homophones(x))
    
    # Generate the output file path
    base_name = os.path.basename(file_path)
    output_file_name = base_name.replace('_test.csv', '_HP.csv')
    output_file_path = os.path.join('MMLU_HP', output_file_name)
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    df.to_csv(output_file_path, index=False)
    print(f"Processed {file_path} and saved results to {output_file_path}")

# List all CSV files in the directory
#all_files = os.listdir(directory)
#csv_files = [file for file in all_files if file.endswith('_test.csv')]
csv_files = ['moral_scenarios_test.csv',
 'us_foreign_policy_test.csv',
 'public_relations_test.csv',
 'global_facts_test.csv',
 'electrical_engineering_test.csv',
 'astronomy_test.csv',
 'business_ethics_test.csv',
 'jurisprudence_test.csv',
 'high_school_chemistry_test.csv',
 'college_physics_test.csv',
 'professional_psychology_test.csv',
 'marketing_test.csv',
 'management_test.csv',
 'virology_test.csv',
 'international_law_test.csv',
 'high_school_macroeconomics_test.csv',
 'prehistory_test.csv',
 'abstract_algebra_test.csv',
 'high_school_physics_test.csv',
 'formal_logic_test.csv',
 'college_medicine_test.csv',
 'high_school_us_history_test.csv',
 'moral_disputes_test.csv',
 'high_school_european_history_test.csv',
 'clinical_knowledge_test.csv',
 'world_religions_test.csv',
 'high_school_microeconomics_test.csv',
 'professional_law_test.csv',
 'human_aging_test.csv',
 'medical_genetics_test.csv',
 'high_school_geography_test.csv',
 'high_school_government_and_politics_test.csv',
 'anatomy_test.csv',
 'sociology_test.csv',
 'logical_fallacies_test.csv',
 'high_school_computer_science_test.csv',
 'miscellaneous_test.csv',
 'high_school_world_history_test.csv',
 'professional_medicine_test.csv',
 'high_school_biology_test.csv',
 'high_school_statistics_test.csv',
 'college_chemistry_test.csv',
 'nutrition_test.csv',
 'econometrics_test.csv',
 'human_sexuality_test.csv',
 'security_studies_test.csv',
 'philosophy_test.csv',
 'elementary_mathematics_test.csv',
 'college_biology_test.csv',
 'college_computer_science_test.csv',
 'machine_learning_test.csv',
 'professional_accounting_test.csv',
 'college_mathematics_test.csv',
 'high_school_mathematics_test.csv',
 'high_school_psychology_test.csv',
 'conceptual_physics_test.csv',
 'computer_security_test.csv']
# Iterate over each CSV file and process it
for file_name in csv_files:
    file_path = os.path.join(directory, file_name)
    process_csv(file_path)

# LeetSpeak

In [ ]:
# general datasets (except MMLU) for leetSpeak
def leet_speak(word):
    leet_dict = {
        'a': ['4', '@', '/\\', 'ä', 'ª'],
        'b': ['8', '|3', 'ß', '13'],
        'c': ['(', '<', '[', '¢'],
        'd': ['[)', '|)', '|>', 'cl'],
        'e': ['3', '€', '&', 'ë'],
        'f': ['|=', 'ph', 'ƒ'],
        'g': ['6', '9', '&'],
        'h': ['#', '[-]', '|-|', '}{'],
        'i': ['1', '!', '|', '][', 'î'],
        'j': ['_|', '_/', ';', '(/'],
        'k': ['|<', '|{', '|(', ']<'],
        'l': ['1', '|_', '|', '£'],
        'm': ['|\\/|', '^^', '/\\/\\', '/V\\'],
        'n': ['|\\|', '/\\/', '^/', '|V'],
        'o': ['0', '()', '*', 'ö'],
        'p': ['|D', '|*', '|o', '|º'],
        'q': ['(,)', '0_', 'O_', '9'],
        'r': ['|2', '|Z', '12', '®'],
        's': ['5', '$', '§'],
        't': ['7', '+', '†'],
        'u': ['|_|', '(_)', 'µ'],
        'v': ['\\/', '|/', '\\|'],
        'w': ['\\/\\/', 'vv', '\\^/', 'uu'],
        'x': ['><', '}{', ')('],
        'y': ['`/', '¥', 'j'],
        'z': ['2', '7_', '%', '>_'],
        'A': ['4', '@', '/\\', 'Ä', 'ª'],
        'B': ['8', '|3', 'ß', '13'],
        'C': ['(', '<', '[', '¢'],
        'D': ['[)', '|)', '|>', 'cl'],
        'E': ['3', '€', '&', 'Ë'],
        'F': ['|=', 'ph', 'ƒ'],
        'G': ['6', '9', '&'],
        'H': ['#', '[-]', '|-|', '}{'],
        'I': ['1', '!', '|', '][', 'Î'],
        'J': ['_|', '_/', ';', '(/'],
        'K': ['|<', '|{', '|(', ']<'],
        'L': ['1', '|_', '|', '£'],
        'M': ['|\\/|', '^^', '/\\/\\', '/V\\'],
        'N': ['|\\|', '/\\/', '^/', '|V'],
        'O': ['0', '()', '*', 'Ö'],
        'P': ['|D', '|*', '|o', '|º'],
        'Q': ['(,)', '0_', 'O_', '9'],
        'R': ['|2', '|Z', '12', '®'],
        'S': ['5', '$', '§'],
        'T': ['7', '+', '†'],
        'U': ['|_|', '(_)', 'µ'],
        'V': ['\\/', '|/', '\\|'],
        'W': ['\\/\\/', 'vv', '\\^/', 'uu'],
        'X': ['><', '}{', ')('],
        'Y': ['`/', '¥', 'j'],
        'Z': ['2', '7_', '%', '>_']
    }

    new_text = []
    for char in word:
        if char in leet_dict:
            new_text.append(random.choice(leet_dict[char]))
        else:
            new_text.append(char)
    return "".join(new_text)

def determine_number_of_words_to_convert(text):
    word_count = len(text.split())
    
    # Example logic to determine number of words based on text length
    if word_count <= 10:
        return 1
    elif word_count <= 20:
        return 2
    elif word_count <= 30:
        return 3
    else:
        return min(5, word_count // 5)  # Convert about 20% of words for longer texts

def random_leet_speak(text):
    if not isinstance(text, str):
        return text  # Return the original value if it's not a string
    words = text.split()
    num_words_to_convert = determine_number_of_words_to_convert(text)
    words_to_convert = random.sample(words, min(num_words_to_convert, len(words)))
    #words_to_convert = random.sample([word for word in words if word != "<mask>"], min(num_words_to_convert, len(words))) # NumerSense only
    
    leeted_text = []
    for word in words:
        if word in words_to_convert:
            leeted_text.append(leet_speak(word))
        else:
            leeted_text.append(word)
    
    return " ".join(leeted_text)

csv_file_path = 'ReClor_RH.csv'  # Replace with your actual file path
df = pd.read_csv(csv_file_path)

# Apply typos to the text column
text_column = 'context'  # Replace with the name of your text column
df['context_Leet'] = df[text_column].apply(lambda x: random_leet_speak(x))
df.to_csv("ReClor_RH.csv", index=False)

In [ ]:
# MMLU for leetSpeak
directory = 'MMLU'

# Function to process a single CSV file
def process_csv(file_path):
    df = pd.read_csv(file_path, header=None, names=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])
    df['Question_Leet'] = df['Question'].apply(lambda x: random_leet_speak(x))
    
    # Generate the output file path
    base_name = os.path.basename(file_path)
    output_file_name = base_name.replace('_test.csv', '_Leet.csv')
    output_file_path = os.path.join('MMLU_Leet', output_file_name)
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    df.to_csv(output_file_path, index=False)
    print(f"Processed {file_path} and saved results to {output_file_path}")

# List all CSV files in the directory
#all_files = os.listdir(directory)
#csv_files = [file for file in all_files if file.endswith('_test.csv')]
csv_files = ['moral_scenarios_test.csv',
 'us_foreign_policy_test.csv',
 'public_relations_test.csv',
 'global_facts_test.csv',
 'electrical_engineering_test.csv',
 'astronomy_test.csv',
 'business_ethics_test.csv',
 'jurisprudence_test.csv',
 'high_school_chemistry_test.csv',
 'college_physics_test.csv',
 'professional_psychology_test.csv',
 'marketing_test.csv',
 'management_test.csv',
 'virology_test.csv',
 'international_law_test.csv',
 'high_school_macroeconomics_test.csv',
 'prehistory_test.csv',
 'abstract_algebra_test.csv',
 'high_school_physics_test.csv',
 'formal_logic_test.csv',
 'college_medicine_test.csv',
 'high_school_us_history_test.csv',
 'moral_disputes_test.csv',
 'high_school_european_history_test.csv',
 'clinical_knowledge_test.csv',
 'world_religions_test.csv',
 'high_school_microeconomics_test.csv',
 'professional_law_test.csv',
 'human_aging_test.csv',
 'medical_genetics_test.csv',
 'high_school_geography_test.csv',
 'high_school_government_and_politics_test.csv',
 'anatomy_test.csv',
 'sociology_test.csv',
 'logical_fallacies_test.csv',
 'high_school_computer_science_test.csv',
 'miscellaneous_test.csv',
 'high_school_world_history_test.csv',
 'professional_medicine_test.csv',
 'high_school_biology_test.csv',
 'high_school_statistics_test.csv',
 'college_chemistry_test.csv',
 'nutrition_test.csv',
 'econometrics_test.csv',
 'human_sexuality_test.csv',
 'security_studies_test.csv',
 'philosophy_test.csv',
 'elementary_mathematics_test.csv',
 'college_biology_test.csv',
 'college_computer_science_test.csv',
 'machine_learning_test.csv',
 'professional_accounting_test.csv',
 'college_mathematics_test.csv',
 'high_school_mathematics_test.csv',
 'high_school_psychology_test.csv',
 'conceptual_physics_test.csv',
 'computer_security_test.csv']
# Iterate over each CSV file and process it
for file_name in csv_files:
    file_path = os.path.join(directory, file_name)
    process_csv(file_path)

# It-cleft

In [ ]:
# general datasets (except MMLU) for It-cleft
nlp = spacy.load("en_core_web_sm")

def it_cleft(sentence):
    doc = nlp(sentence)
    subject = None
    verb = None
    aux = None
    obj = None
    other_tokens = []

    for token in doc:
        if token.dep_ == "nsubj":
            subject = token
        elif token.dep_ == "ROOT":
            verb = token
        elif token.dep_ in ("dobj", "attr"):
            obj = token
        elif token.dep_ == "aux":
            aux = token
        else:
            other_tokens.append(token.text)
    
    if subject and verb:
        other_text = " ".join(other_tokens).strip()
        if obj:
            if aux:
                return f"It was {subject.text} that {aux.text} {verb.text} {obj.text} {other_text}".strip()
            else:
                return f"It was {subject.text} that {verb.text} {obj.text} {other_text}".strip()
        else:
            if aux:
                return f"It was {subject.text} that {aux.text} {verb.text} {other_text}".strip()
            else:
                return f"It was {subject.text} that {verb.text} {other_text}".strip()
    return sentence

def process_sentences(sentence):
    if not isinstance(sentence, str):
        return sentence
    # Split by sentence boundaries
    doc = nlp(sentence)
    sentences = [sent.text for sent in doc.sents]
    cleft_sentences = [it_cleft(sent) for sent in sentences]
    return ' '.join(cleft_sentences)

csv_file_path = 'ReClor_RH.csv'  # Replace with your actual file path
df = pd.read_csv(csv_file_path)

# Apply typos to the text column
text_column = 'context'
df['context_It_cleft'] = df[text_column].apply(lambda x: process_sentences(x))
df.to_csv("ReClor_RH.csv", index=False)

In [ ]:
# MMLU for It-cleft
directory = 'MMLU'

# Function to process a single CSV file
def process_csv(file_path):
    df = pd.read_csv(file_path, header=None, names=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])
    df['Question_It_cleft'] = df['Question'].apply(lambda x: process_sentences(x))
    
    # Generate the output file path
    base_name = os.path.basename(file_path)
    output_file_name = base_name.replace('_test.csv', '_It_cleft.csv')
    output_file_path = os.path.join('MMLU_It_cleft', output_file_name)
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    df.to_csv(output_file_path, index=False)
    print(f"Processed {file_path} and saved results to {output_file_path}")

# List all CSV files in the directory
#all_files = os.listdir(directory)
#csv_files = [file for file in all_files if file.endswith('_test.csv')]
csv_files = ['moral_scenarios_test.csv',
 'us_foreign_policy_test.csv',
 'public_relations_test.csv',
 'global_facts_test.csv',
 'electrical_engineering_test.csv',
 'astronomy_test.csv',
 'business_ethics_test.csv',
 'jurisprudence_test.csv',
 'high_school_chemistry_test.csv',
 'college_physics_test.csv',
 'professional_psychology_test.csv',
 'marketing_test.csv',
 'management_test.csv',
 'virology_test.csv',
 'international_law_test.csv',
 'high_school_macroeconomics_test.csv',
 'prehistory_test.csv',
 'abstract_algebra_test.csv',
 'high_school_physics_test.csv',
 'formal_logic_test.csv',
 'college_medicine_test.csv',
 'high_school_us_history_test.csv',
 'moral_disputes_test.csv',
 'high_school_european_history_test.csv',
 'clinical_knowledge_test.csv',
 'world_religions_test.csv',
 'high_school_microeconomics_test.csv',
 'professional_law_test.csv',
 'human_aging_test.csv',
 'medical_genetics_test.csv',
 'high_school_geography_test.csv',
 'high_school_government_and_politics_test.csv',
 'anatomy_test.csv',
 'sociology_test.csv',
 'logical_fallacies_test.csv',
 'high_school_computer_science_test.csv',
 'miscellaneous_test.csv',
 'high_school_world_history_test.csv',
 'professional_medicine_test.csv',
 'high_school_biology_test.csv',
 'high_school_statistics_test.csv',
 'college_chemistry_test.csv',
 'nutrition_test.csv',
 'econometrics_test.csv',
 'human_sexuality_test.csv',
 'security_studies_test.csv',
 'philosophy_test.csv',
 'elementary_mathematics_test.csv',
 'college_biology_test.csv',
 'college_computer_science_test.csv',
 'machine_learning_test.csv',
 'professional_accounting_test.csv',
 'college_mathematics_test.csv',
 'high_school_mathematics_test.csv',
 'high_school_psychology_test.csv',
 'conceptual_physics_test.csv',
 'computer_security_test.csv']
# Iterate over each CSV file and process it
for file_name in csv_files:
    file_path = os.path.join(directory, file_name)
    process_csv(file_path)

# Wh-cleft

In [ ]:
# general datasets (except MMLU) for Wh-cleft

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

def wh_cleft(sentence):
    doc = nlp(sentence)
    subject = None
    verb = None
    aux = None
    obj = None
    other_tokens = []
    wh_word = "what"  # Default to "what"

    for token in doc:
        if token.dep_ == "nsubj":
            subject = token
        elif token.dep_ == "ROOT":
            verb = token
        elif token.dep_ in ("dobj", "attr"):
            obj = token
        elif token.dep_ == "aux":
            aux = token
        else:
            other_tokens.append(token.text)
    
    if subject and verb:
        other_text = " ".join(other_tokens).strip()
        
        # Determine the appropriate wh- word
        if subject.ent_type_ in ["PERSON", "ORG"]:
            wh_word = "who"
        elif "time" in other_text.lower() or "when" in other_text.lower():
            wh_word = "when"
        elif "place" in other_text.lower() or "where" in other_text.lower():
            wh_word = "where"
        elif "reason" in other_text.lower() or "why" in other_text.lower():
            wh_word = "why"
        elif "manner" in other_text.lower() or "how" in other_text.lower():
            wh_word = "how"
        elif subject.ent_type_ in ["GPE", "LOC"]:
            wh_word = "where"
        else:
            wh_word = "what"
        
        if obj:
            if aux:
                return f"{wh_word.capitalize()} {aux.text} {verb.text} {obj.text} {other_text} was {subject.text}".strip()
            else:
                return f"{wh_word.capitalize()} {verb.text} {obj.text} {other_text} was {subject.text}".strip()
        else:
            if aux:
                return f"{wh_word.capitalize()} {aux.text} {verb.text} {other_text} was {subject.text}".strip()
            else:
                return f"{wh_word.capitalize()} {verb.text} {other_text} was {subject.text}".strip()
    return sentence

def process_sentences(sentence):
    if not isinstance(sentence, str):
        return sentence
    doc = nlp(sentence)
    sentences = [sent.text for sent in doc.sents]
    cleft_sentences = [wh_cleft(sent) for sent in sentences]
    return ' '.join(cleft_sentences)

csv_file_path = 'ReClor_RH.csv'  # Replace with your actual file path
df = pd.read_csv(csv_file_path)

# Apply typos to the text column
text_column = 'context'
df['context_Wh_cleft'] = df[text_column].apply(lambda x: process_sentences(x))
df.to_csv("ReClor_RH.csv", index=False)

In [ ]:
# MMLU for Wh-cleft
directory = 'MMLU'

# Function to process a single CSV file
def process_csv(file_path):
    df = pd.read_csv(file_path, header=None, names=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])
    df['Question_Wh_cleft'] = df['Question'].apply(lambda x: process_sentences(x))
    
    # Generate the output file path
    base_name = os.path.basename(file_path)
    output_file_name = base_name.replace('_test.csv', '_Wh_cleft.csv')
    output_file_path = os.path.join('MMLU_Wh_cleft', output_file_name)
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    df.to_csv(output_file_path, index=False)
    print(f"Processed {file_path} and saved results to {output_file_path}")

# List all CSV files in the directory
#all_files = os.listdir(directory)
#csv_files = [file for file in all_files if file.endswith('_test.csv')]
csv_files = ['moral_scenarios_test.csv',
 'us_foreign_policy_test.csv',
 'public_relations_test.csv',
 'global_facts_test.csv',
 'electrical_engineering_test.csv',
 'astronomy_test.csv',
 'business_ethics_test.csv',
 'jurisprudence_test.csv',
 'high_school_chemistry_test.csv',
 'college_physics_test.csv',
 'professional_psychology_test.csv',
 'marketing_test.csv',
 'management_test.csv',
 'virology_test.csv',
 'international_law_test.csv',
 'high_school_macroeconomics_test.csv',
 'prehistory_test.csv',
 'abstract_algebra_test.csv',
 'high_school_physics_test.csv',
 'formal_logic_test.csv',
 'college_medicine_test.csv',
 'high_school_us_history_test.csv',
 'moral_disputes_test.csv',
 'high_school_european_history_test.csv',
 'clinical_knowledge_test.csv',
 'world_religions_test.csv',
 'high_school_microeconomics_test.csv',
 'professional_law_test.csv',
 'human_aging_test.csv',
 'medical_genetics_test.csv',
 'high_school_geography_test.csv',
 'high_school_government_and_politics_test.csv',
 'anatomy_test.csv',
 'sociology_test.csv',
 'logical_fallacies_test.csv',
 'high_school_computer_science_test.csv',
 'miscellaneous_test.csv',
 'high_school_world_history_test.csv',
 'professional_medicine_test.csv',
 'high_school_biology_test.csv',
 'high_school_statistics_test.csv',
 'college_chemistry_test.csv',
 'nutrition_test.csv',
 'econometrics_test.csv',
 'human_sexuality_test.csv',
 'security_studies_test.csv',
 'philosophy_test.csv',
 'elementary_mathematics_test.csv',
 'college_biology_test.csv',
 'college_computer_science_test.csv',
 'machine_learning_test.csv',
 'professional_accounting_test.csv',
 'college_mathematics_test.csv',
 'high_school_mathematics_test.csv',
 'high_school_psychology_test.csv',
 'conceptual_physics_test.csv',
 'computer_security_test.csv']
# Iterate over each CSV file and process it
for file_name in csv_files:
    file_path = os.path.join(directory, file_name)
    process_csv(file_path)

# StressTest

In [ ]:
# general datasets (except MMLU) for StressTest
stress_statements = [
    "and true is true",
    "and false is not true",
    "and true is true",
    "if one is equal to one",
    "and the sky is blue",
    "if water is wet",
    "and two plus two is four",
    "if the earth orbits the sun",
    "and fire is hot",
    "if gravity pulls objects down"
]

def create_stress_test(statements, repetitions=5):
    return " ".join(random.choices(statements, k=repetitions))

def insert_stress_test_randomly(sentence, statements, repetitions=5):
    if not isinstance(sentence, str):
        return sentence
    stress_test = create_stress_test(statements, repetitions)
    words = sentence.split()
    stress_test_words = stress_test.split()
    
    if len(stress_test_words) >= len(words):
        # Append stress test at the end if not enough words to insert randomly
        return f"{sentence} {stress_test}"
    
    # Determine random insertion points
    insert_points = sorted(random.sample(range(len(words) + 1), len(stress_test_words)))

    # Insert stress test words at the determined points
    perturbed_sentence = []
    stress_test_index = 0
    for i in range(len(words) + len(stress_test_words)):
        if i in insert_points and stress_test_index < len(stress_test_words):
            perturbed_sentence.append(stress_test_words[stress_test_index])
            stress_test_index += 1
        else:
            if words:
                perturbed_sentence.append(words.pop(0))
    
    return ' '.join(perturbed_sentence)

csv_file_path = 'ReClor_RH.csv'  # Replace with your actual file path
df = pd.read_csv(csv_file_path)

# Apply typos to the text column
text_column = 'context'
df['context_stress'] = df[text_column].apply(lambda x: insert_stress_test_randomly(x, stress_statements, 5))

# Save the DataFrame to a new CSV file (optional)
df.to_csv('ReClor_RH.csv', index=False)

In [ ]:
# MMLU for StressTest
directory = 'MMLU'

# Function to process a single CSV file
def process_csv(file_path):
    df = pd.read_csv(file_path, header=None, names=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])
    df['Question_stress'] = df['Question'].apply(lambda x: insert_stress_test_randomly(x, stress_statements, 5))
    
    # Generate the output file path
    base_name = os.path.basename(file_path)
    output_file_name = base_name.replace('_test.csv', '_stress.csv')
    output_file_path = os.path.join('MMLU_stress', output_file_name)
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    df.to_csv(output_file_path, index=False)
    print(f"Processed {file_path} and saved results to {output_file_path}")

# List all CSV files in the directory
#all_files = os.listdir(directory)
#csv_files = [file for file in all_files if file.endswith('_test.csv')]
csv_files = ['moral_scenarios_test.csv',
 'us_foreign_policy_test.csv',
 'public_relations_test.csv',
 'global_facts_test.csv',
 'electrical_engineering_test.csv',
 'astronomy_test.csv',
 'business_ethics_test.csv',
 'jurisprudence_test.csv',
 'high_school_chemistry_test.csv',
 'college_physics_test.csv',
 'professional_psychology_test.csv',
 'marketing_test.csv',
 'management_test.csv',
 'virology_test.csv',
 'international_law_test.csv',
 'high_school_macroeconomics_test.csv',
 'prehistory_test.csv',
 'abstract_algebra_test.csv',
 'high_school_physics_test.csv',
 'formal_logic_test.csv',
 'college_medicine_test.csv',
 'high_school_us_history_test.csv',
 'moral_disputes_test.csv',
 'high_school_european_history_test.csv',
 'clinical_knowledge_test.csv',
 'world_religions_test.csv',
 'high_school_microeconomics_test.csv',
 'professional_law_test.csv',
 'human_aging_test.csv',
 'medical_genetics_test.csv',
 'high_school_geography_test.csv',
 'high_school_government_and_politics_test.csv',
 'anatomy_test.csv',
 'sociology_test.csv',
 'logical_fallacies_test.csv',
 'high_school_computer_science_test.csv',
 'miscellaneous_test.csv',
 'high_school_world_history_test.csv',
 'professional_medicine_test.csv',
 'high_school_biology_test.csv',
 'high_school_statistics_test.csv',
 'college_chemistry_test.csv',
 'nutrition_test.csv',
 'econometrics_test.csv',
 'human_sexuality_test.csv',
 'security_studies_test.csv',
 'philosophy_test.csv',
 'elementary_mathematics_test.csv',
 'college_biology_test.csv',
 'college_computer_science_test.csv',
 'machine_learning_test.csv',
 'professional_accounting_test.csv',
 'college_mathematics_test.csv',
 'high_school_mathematics_test.csv',
 'high_school_psychology_test.csv',
 'conceptual_physics_test.csv',
 'computer_security_test.csv']
# Iterate over each CSV file and process it
for file_name in csv_files:
    file_path = os.path.join(directory, file_name)
    process_csv(file_path)

# CheckList

In [ ]:
# general datasets (except MMLU) for CheckList
random_urls_handles = [
    "http://google.com",
    "https://github.com",
    "@realuser1",
    "@tech_guru",
    "https://bit.ly/3uA1abc",
    "http://tinyurl.com/real-example",
    "@famous_person",
    "https://short.url/tech",
    "http://example.org/news",
    "@cool_handle",
    "http://stackoverflow.com",
    "https://twitter.com/realuser",
    "@developer123",
    "@random_tech",
    "https://news.ycombinator.com",
    "http://nytimes.com",
    "https://linkedin.com/in/someone",
    "@official_account",
    "@news_update",
    "https://medium.com/@writer",
    "http://bbc.com",
    "https://cnn.com",
    "@follower1",
    "@social_media",
    "https://t.co/xyz123",
    "http://espn.com",
    "https://facebook.com/profile",
    "@techie_life",
    "@daily_news",
    "http://instagram.com",
    "https://youtube.com",
    "@content_creator",
    "https://pinterest.com",
    "http://reddit.com",
    "@trending_now",
    "https://snapchat.com",
    "http://whatsapp.com",
    "@messenger_app",
    "https://tiktok.com",
    "http://quora.com",
    "@ask_me_anything",
    "https://twitch.tv",
    "http://vimeo.com",
    "@video_stream",
    "https://flickr.com",
    "http://tumblr.com",
    "@blogger_handle",
    "https://wordpress.com",
    "http://medium.com",
    "@writing_hub",
    "https://producthunt.com",
    "http://angel.co",
    "@startup_founder",
    "https://crunchbase.com",
    "http://gizmodo.com",
    "@tech_news",
    "https://mashable.com",
    "http://engadget.com",
    "@gadget_guru",
    "https://techcrunch.com",
    "http://theverge.com",
    "@tech_updates",
    "https://wired.com",
    "http://cnet.com",
    "@tech_reviews",
    "https://recode.net",
    "http://vox.com",
    "@media_outlet",
    "https://buzzfeed.com",
    "http://huffpost.com",
    "@news_aggregator",
    "https://forbes.com",
    "http://bloomberg.com",
    "@finance_news",
    "https://economist.com",
    "http://wsj.com",
    "@market_updates",
    "https://fortune.com",
    "http://marketwatch.com",
    "@stock_tips",
    "https://seekingalpha.com",
    "http://investopedia.com",
    "@financial_guru",
    "https://cnbc.com",
    "http://reuters.com",
    "@newswire",
    "https://apnews.com",
    "http://npr.org",
    "@public_radio",
    "https://bbc.co.uk",
    "http://aljazeera.com",
    "@global_news",
    "https://dw.com",
    "http://france24.com",
    "@world_news",
    "https://rt.com",
    "http://sputniknews.com",
    "@news_russia",
    "https://japantimes.co.jp",
    "http://chinadaily.com.cn",
    "@news_asia"
]

def create_random_elements(elements, repetitions=5):
    return " ".join(random.choices(elements, k=repetitions))

def insert_random_elements_randomly(sentence, elements, repetitions=5):
    if not isinstance(sentence, str):
        return sentence
    random_elements = create_random_elements(elements, repetitions)
    words = sentence.split()
    random_elements_words = random_elements.split()
    
    if len(random_elements_words) >= len(words):
        # Append random elements at the end if not enough words to insert randomly
        return f"{sentence} {random_elements}"
    
    # Determine random insertion points
    insert_points = sorted(random.sample(range(len(words) + 1), len(random_elements_words)))

    # Insert random elements at the determined points
    perturbed_sentence = []
    random_elements_index = 0
    for i in range(len(words) + len(random_elements_words)):
        if i in insert_points and random_elements_index < len(random_elements_words):
            perturbed_sentence.append(random_elements_words[random_elements_index])
            random_elements_index += 1
        else:
            if words:
                perturbed_sentence.append(words.pop(0))
    
    return ' '.join(perturbed_sentence)

csv_file_path = 'ReClor_RH.csv'  # Replace with your actual file path
df = pd.read_csv(csv_file_path)

# Apply typos to the text column
text_column = 'context'
df['context_checklist'] = df[text_column].apply(lambda x: insert_random_elements_randomly(x, random_urls_handles, 5))

# Save the DataFrame to a new CSV file (optional)
df.to_csv('ReClor_RH.csv', index=False)

In [ ]:
# MMLU for CheckList

directory = 'MMLU'

# Function to process a single CSV file
def process_csv(file_path):
    df = pd.read_csv(file_path, header=None, names=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])
    df['Question_checklist'] = df['Question'].apply(lambda x: insert_random_elements_randomly(x, random_urls_handles, 5))
    
    # Generate the output file path
    base_name = os.path.basename(file_path)
    output_file_name = base_name.replace('_test.csv', '_checklist.csv')
    output_file_path = os.path.join('MMLU_checklist', output_file_name)
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    df.to_csv(output_file_path, index=False)
    print(f"Processed {file_path} and saved results to {output_file_path}")

# List all CSV files in the directory
#all_files = os.listdir(directory)
#csv_files = [file for file in all_files if file.endswith('_test.csv')]
csv_files = ['moral_scenarios_test.csv',
 'us_foreign_policy_test.csv',
 'public_relations_test.csv',
 'global_facts_test.csv',
 'electrical_engineering_test.csv',
 'astronomy_test.csv',
 'business_ethics_test.csv',
 'jurisprudence_test.csv',
 'high_school_chemistry_test.csv',
 'college_physics_test.csv',
 'professional_psychology_test.csv',
 'marketing_test.csv',
 'management_test.csv',
 'virology_test.csv',
 'international_law_test.csv',
 'high_school_macroeconomics_test.csv',
 'prehistory_test.csv',
 'abstract_algebra_test.csv',
 'high_school_physics_test.csv',
 'formal_logic_test.csv',
 'college_medicine_test.csv',
 'high_school_us_history_test.csv',
 'moral_disputes_test.csv',
 'high_school_european_history_test.csv',
 'clinical_knowledge_test.csv',
 'world_religions_test.csv',
 'high_school_microeconomics_test.csv',
 'professional_law_test.csv',
 'human_aging_test.csv',
 'medical_genetics_test.csv',
 'high_school_geography_test.csv',
 'high_school_government_and_politics_test.csv',
 'anatomy_test.csv',
 'sociology_test.csv',
 'logical_fallacies_test.csv',
 'high_school_computer_science_test.csv',
 'miscellaneous_test.csv',
 'high_school_world_history_test.csv',
 'professional_medicine_test.csv',
 'high_school_biology_test.csv',
 'high_school_statistics_test.csv',
 'college_chemistry_test.csv',
 'nutrition_test.csv',
 'econometrics_test.csv',
 'human_sexuality_test.csv',
 'security_studies_test.csv',
 'philosophy_test.csv',
 'elementary_mathematics_test.csv',
 'college_biology_test.csv',
 'college_computer_science_test.csv',
 'machine_learning_test.csv',
 'professional_accounting_test.csv',
 'college_mathematics_test.csv',
 'high_school_mathematics_test.csv',
 'high_school_psychology_test.csv',
 'conceptual_physics_test.csv',
 'computer_security_test.csv']
# Iterate over each CSV file and process it
for file_name in csv_files:
    file_path = os.path.join(directory, file_name)
    process_csv(file_path)

# Compound Variations

In [ ]:
# general datasets for compound variations (except NumerSense and MMLU)
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

def complex_sentence_variation(sentence):
    if not isinstance(sentence, str):
        return sentence  
    words = nltk.word_tokenize(sentence)
    pos_tags = nltk.pos_tag(words)

    new_sentence = sentence

    # Expanded lists of subordinating conjunctions and quantifiers
    conjunctions = [
        "although", "because", "since", "when", "while", "after", "before", 
        "if", "even though", "unless", "whereas", "though", "provided that", 
        "as long as", "in case", "until", "as soon as", "now that", "once"
    ]
    quantifiers = [
        "some", "many", "several", "few", "a lot of", "a couple of", 
        "numerous", "plenty of", "various", "a handful of", "an abundance of", 
        "a number of", "much", "more", "less", "a majority of", "a minority of", 
        "sufficient", "all", "no"
    ]

    # Add subordinating conjunctions to create complex sentences
    chosen_conj = random.choice(conjunctions)
    if chosen_conj not in new_sentence:
        new_sentence = chosen_conj.capitalize() + " " + new_sentence

    # Add quantifiers without touching numbers
    for i, (word, pos) in enumerate(pos_tags):
        if word.isdigit():
            continue
        if pos in ['NN', 'NNS'] and not word.isdigit():  # Singular and plural nouns
            new_sentence = new_sentence.replace(word, random.choice(quantifiers) + " " + word, 1)
            break

    # Modify Punctuation
    words = nltk.word_tokenize(new_sentence)
    if len(words) > 2:
        insert_pos = random.randint(1, len(words) - 2)
        words.insert(insert_pos, ',')
        new_sentence = " ".join(words)

    return new_sentence

csv_file_path = 'ReClor_RH.csv'  # Replace with your actual file path
df = pd.read_csv(csv_file_path)

# Apply typos to the text column
text_column = 'context'
df['context_comp'] = df[text_column].apply(lambda x: complex_sentence_variation(x))

# Save the DataFrame to a new CSV file (optional)
df.to_csv('ReClor_RH.csv', index=False)

In [ ]:
# NumerSense for compound variations (do not move the <mask> in the text)
def complex_sentence_variation(sentence):
    if not isinstance(sentence, str):
        return sentence

    # Split the sentence around <mask> tokens and process each segment separately
    segments = sentence.split('<mask>')
    new_segments = []

    for i, segment in enumerate(segments):
        if i > 0:  # Re-add the <mask> token at the beginning of each segment except the first
            new_segments.append('<mask>')

        words = nltk.word_tokenize(segment)
        pos_tags = nltk.pos_tag(words)

        # Expanded lists of subordinating conjunctions and quantifiers
        conjunctions = [
            "although", "because", "since", "when", "while", "after", "before", 
            "if", "even though", "unless", "whereas", "though", "provided that", 
            "as long as", "in case", "until", "as soon as", "now that", "once"
        ]
        quantifiers = [
            "some", "many", "several", "few", "a lot of", "a couple of", 
            "numerous", "plenty of", "various", "a handful of", "an abundance of", 
            "a number of", "much", "more", "less", "a majority of", "a minority of", 
            "sufficient", "all", "no"
        ]

        # Add subordinating conjunctions to create complex sentences
        if segment.strip():
            chosen_conj = random.choice(conjunctions)
            segment = chosen_conj.capitalize() + " " + segment

            # Add quantifiers without touching numbers
            for word, pos in pos_tags:
                if word.isdigit():
                    continue
                if pos in ['NN', 'NNS'] and not word.isdigit():  # Singular and plural nouns
                    segment = segment.replace(word, random.choice(quantifiers) + " " + word, 1)
                    break

            # Modify Punctuation
            words = nltk.word_tokenize(segment)
            if len(words) > 2:
                insert_pos = random.randint(1, len(words) - 2)
                words.insert(insert_pos, ',')
                segment = " ".join(words)

        new_segments.append(segment)

    new_sentence = ''.join(new_segments)
    return new_sentence

In [ ]:
# MMLU for compound variations
directory = 'MMLU'

# Function to process a single CSV file
def process_csv(file_path):
    df = pd.read_csv(file_path, header=None, names=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])
    df['Question_comp'] = df['Question'].apply(lambda x: complex_sentence_variation(x))
    
    # Generate the output file path
    base_name = os.path.basename(file_path)
    output_file_name = base_name.replace('_test.csv', '_comp.csv')
    output_file_path = os.path.join('MMLU_comp', output_file_name)
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    df.to_csv(output_file_path, index=False)
    print(f"Processed {file_path} and saved results to {output_file_path}")

# List all CSV files in the directory
#all_files = os.listdir(directory)
#csv_files = [file for file in all_files if file.endswith('_test.csv')]
csv_files = ['moral_scenarios_test.csv',
 'us_foreign_policy_test.csv',
 'public_relations_test.csv',
 'global_facts_test.csv',
 'electrical_engineering_test.csv',
 'astronomy_test.csv',
 'business_ethics_test.csv',
 'jurisprudence_test.csv',
 'high_school_chemistry_test.csv',
 'college_physics_test.csv',
 'professional_psychology_test.csv',
 'marketing_test.csv',
 'management_test.csv',
 'virology_test.csv',
 'international_law_test.csv',
 'high_school_macroeconomics_test.csv',
 'prehistory_test.csv',
 'abstract_algebra_test.csv',
 'high_school_physics_test.csv',
 'formal_logic_test.csv',
 'college_medicine_test.csv',
 'high_school_us_history_test.csv',
 'moral_disputes_test.csv',
 'high_school_european_history_test.csv',
 'clinical_knowledge_test.csv',
 'world_religions_test.csv',
 'high_school_microeconomics_test.csv',
 'professional_law_test.csv',
 'human_aging_test.csv',
 'medical_genetics_test.csv',
 'high_school_geography_test.csv',
 'high_school_government_and_politics_test.csv',
 'anatomy_test.csv',
 'sociology_test.csv',
 'logical_fallacies_test.csv',
 'high_school_computer_science_test.csv',
 'miscellaneous_test.csv',
 'high_school_world_history_test.csv',
 'professional_medicine_test.csv',
 'high_school_biology_test.csv',
 'high_school_statistics_test.csv',
 'college_chemistry_test.csv',
 'nutrition_test.csv',
 'econometrics_test.csv',
 'human_sexuality_test.csv',
 'security_studies_test.csv',
 'philosophy_test.csv',
 'elementary_mathematics_test.csv',
 'college_biology_test.csv',
 'college_computer_science_test.csv',
 'machine_learning_test.csv',
 'professional_accounting_test.csv',
 'college_mathematics_test.csv',
 'high_school_mathematics_test.csv',
 'high_school_psychology_test.csv',
 'conceptual_physics_test.csv',
 'computer_security_test.csv']
# Iterate over each CSV file and process it
for file_name in csv_files:
    file_path = os.path.join(directory, file_name)
    process_csv(file_path)